<a href="https://colab.research.google.com/github/r0brt/Daan_Semesterarbeit/blob/master/HaemmerliRobert_Daan_QLearning_FrozenLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Frozen Lake
![alt text](http://deeplizard.com/images/frozen%20lake%20winter.jpg)

### Setting up Frozen Lake


#### Libraries

In [0]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

#### Creating the environment

Next, to create our environment, we just call gym.make() and pass a string of the name of the environment we want to set up. We'll be using the environment FrozenLake-v0. All the environments with their corresponding names you can use here are available on Gym’s website https://gym.openai.com/envs/#classic_control

In [0]:
env = gym.make("FrozenLake-v0")

#### Creating the Q-table

We’re now going to construct our Q-table, and initialize all the Q-values to zero for each state-action pair.

Remember, the number of rows in the table is equivalent to the size of the state space in the environment, and the number of columns is equivalent to the size of the action space. We can get this information using using env.observation_space.n and env.action_space.n, as shown below. We can then use this information to build the Q-table and fill it with zeros. 


In [0]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [0]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


#### Initializing Q-learning parameters

First, with **num_episodes**, we define the total number of episodes we want the agent to play during training. Then, with **max_steps_per_episode**, we define a maximum number of steps that our agent is allowed to take within a single episode. So, if by the one-hundredth step, the agent hasn’t reached the frisbee or fallen through a hole, then the episode will terminate with the agent receiving zero points.

Next, we set our **learning_rate**, which was mathematically shown using the symbol α
in the previous post. Then, we also set our **discount_rate**, as well, which was represented with the symbol γ previously.

Now, the last four parameters are all for related to the exploration-exploitation trade-off we talked about last time in regards to the epsilon-greedy po licy. We’re initializing our **exploration_rate** to 1 and setting the **max_exploration_rate** to 1 and a **min_exploration_rate** to 0.01. The max and min are just bounds to how large or small our exploration rate can be.
Remember, the exploration rate was represented with the symbol ϵwhen we discussed it previously.

Lastly, we set the **exploration_decay_rate** to 0.01 to determine the rate at which the exploration_rate will decay. 

In [0]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.01
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

#### The Q-learning algorithm training loop 

`Update Q-table for Q(s,a)`:  
The new Q-value for this state-action pair is a weighted sum of our old value and the “learned value.” So we have our new Q-value equal to the old Q-value times one minus the learning rate plus the learning rate times the “learned value,” which is the reward we just received from our last action plus the discounted estimate of the optimal future Q-value for the next state action pair.

In [0]:
# list to hold all of the rewards
rewards_all_episodes = []

# Q-learning algorithm
for episode in range(num_episodes):
    # initialize new episode params
    state = env.reset()
    done = False                 # keeps track of whether or not our episode is finished
    rewards_current_episode = 0  # keep track of the rewards
    
    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:  # exploit the environment and choose the action that has the highest Q-value
            action = np.argmax(q_table[state,:]) 
        else:                                              # agent will explore the environment, and sample an action randomly
            action = env.action_space.sample()
            
        # Take new action
        new_state, reward, done, info = env.step(action)
        
        # Update Q-table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
                
        # Set new state
        state = new_state
        
        # Add new reward   
        rewards_current_episode += reward
        
        if done == True: 
            break


    # Exploration rate decay
    # Exploration rate decreases or decays at a rate proportional to its current value
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode) 
    
    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)

In [44]:
# Calculate and print the average reward per thousand episodes
rewards_per_thosand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thosand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.05600000000000004
2000 :  0.17900000000000013
3000 :  0.43300000000000033
4000 :  0.5820000000000004
5000 :  0.6470000000000005
6000 :  0.6420000000000005
7000 :  0.6740000000000005
8000 :  0.6960000000000005
9000 :  0.6890000000000005
10000 :  0.7110000000000005


In [25]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)



********Q-table********

[[0.49285598 0.48567299 0.48430922 0.48481107]
 [0.37695337 0.36177207 0.25163424 0.47018398]
 [0.41114561 0.41787378 0.41462325 0.44898486]
 [0.29468777 0.40065272 0.29184702 0.43418967]
 [0.51973289 0.46145668 0.41746398 0.3182259 ]
 [0.         0.         0.         0.        ]
 [0.20821688 0.15222062 0.40475105 0.16150516]
 [0.         0.         0.         0.        ]
 [0.41552005 0.28980449 0.33176444 0.56930379]
 [0.44063289 0.63271233 0.38909196 0.26649917]
 [0.65579346 0.39797181 0.44981905 0.35714035]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.48279929 0.42402213 0.66291352 0.47703417]
 [0.72336321 0.82099667 0.75904861 0.75330241]
 [0.         0.         0.         0.        ]]


#### Watch the agent play the game

In [45]:
# Watch our agent play Frozen Lake by playing the best action 
# from each state according to the Q-table

for episode in range(3):
    # initialize new episode params
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)

    for step in range(max_steps_per_episode):        
        # Show current state of environment on screen    
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        
        # Choose action with highest Q-value for current state  
        action = np.argmax(q_table[state,:])   
        
        # Take new action
        new_state, reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                # Agent reached the goal and won episode
                print("****You reached the goal!****")
                time.sleep(3)                
            else:
                # Agent stepped in a hole and lost episode      
                print("****You fell through a hole!****")
                time.sleep(3)
                clear_output(wait=True)
            break
            
        # Set new state
        state = new_state
        
env.close()

  (Right)
SFFF
FHFH
FFFH
HFFG
****You fell through a hole!****
